In [87]:
# instal lthe following modules:
# beautifulsoup4 for web scraping of the wikipedia page
# lxml, html5lib for perper conversion with beautifulsoup4
# For coordinates: latitudes and longitudes

!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install geopy
print('BeautifulSoup, lxml and html5lib, geopy installed.')

     |████████████████████████████████| 102kB 10.6MB/s ta 0:00:01
BeautifulSoup, lxml and html5lib installed.


In [119]:
import numpy as np
import pandas as pd # library for data analsysis
import matplotlib as mplib
import csv
import requests
import sys
import json
import urllib3
import folium
import lxml
import html5lib
import folium
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [120]:
# setting display to display the entire dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [121]:
# import Wikipedia URL
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# print(url)

In [122]:
# soup = BeautifulSoup(url,features="xml") # did not work
soup = BeautifulSoup(url, "html.parser")
table = soup.find('table',{'class':'wikitable sortable'})
# print(soup.prettify())

In [123]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pc.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [124]:
# Remove "Not assigned" and then Aggregate
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [128]:
# df_pc # # line 8: Queen's Park : Not assigned
#df_pc

In [129]:
# Queen's Park is not assigned: make the neighborhood same as the borough (in general: do for all Neighbourhood not assigned, Neighbourhood = Borough)
df_pc['Neighbourhood'].replace("Not assigned","Queen's Park",inplace=True)

In [131]:
# df_pc

In [132]:
df_pc.shape
# the same postcode may have more than one neighbourhood

(211, 3)

In [133]:
# using groupby on Neighbourhood and applying an inner join to group together the neighbourhood avec le meme code postale
df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pcn.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [134]:
df_pcn.shape

(103, 3)

In [106]:
# df_pcn

In [135]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [136]:
df_pos = pd.merge(df_pcn, df_geo, on=['Postcode'], how='inner')
df_tor = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
df_tor.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [137]:
# df_tor

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


In [138]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [142]:
# filter all Borough containing they key Toronto
df_tor = df_tor[df_tor['Borough'].str.contains('Toronto', na = False)]


In [143]:
df_tor

,Borough,Neighbourhood,Postcode,Latitude,Longitude
37,East Toronto,The Beaches,M4E,43.676357,-79.293031
41,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
42,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
43,East Toronto,Studio District,M4M,43.659526,-79.340923
44,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
45,Central Toronto,Davisville North,M4P,43.712751,-79.390197
46,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
47,Central Toronto,Davisville,M4S,43.704324,-79.388790
48,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
49,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V,43.686412,-79.400049


In [144]:
# create map of Toronto using latitude and longitude values (only Borough containing Toronto )
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto